## Assignment 2: Set up the work for data-driven inference based on CitiBike data
###### Author: Daniel Fay
   * Write a Jupyter Notebook on compute. This will require you to use the JupyterHub (instructions here ). Write a notebook thats states the question you want to ask, and formulates the Null and Alternative hypothesis (remember the confidence level!)
   * Use pandas to read in the CitiBike files, either from the DF, or locally, but you must be able to download them on the spot (so the TA can reproduce your work).
   * Display the top few rows of the DF in your notebook. This table must be rendered.
   * Display the reducted dataframe. This table must be rendered.
   * Plot your data distributions.